In [1]:
import anndata as ad 

In [2]:
adata = ad.read('resources/grn_benchmark//inference_data/op_atac.h5ad')

/home/jnourisa/miniconda3/envs/py10/lib/python3.10/site-packages/anndata/__init__.py:52: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


In [4]:
adata.var

,seqname,ranges,strand
location,,,
chr1:9785-10672,chr1,9785-10672,+
chr1:180537-181293,chr1,180537-181293,+
chr1:183952-184816,chr1,183952-184816,+
chr1:191200-192081,chr1,191200-192081,+
chr1:267572-268456,chr1,267572-268456,+
...,...,...,...
KI270713.1:3945-4846,KI270713.1,3945-4846,+
KI270713.1:21464-22377,KI270713.1,21464-22377,+
KI270713.1:27714-28535,KI270713.1,27714-28535,+
